In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from openpyxl.utils import get_column_letter
from openpyxl import Workbook
import sqlite3
import sqlite3

In [ ]:
# Load data from excel, and explore the data

#raw_data = pd.read_excel('2023_01_01_ExportRechLibre.xlsx', engine='openpyxl')
file_path = '2023_01_01_ExportRechLibre.xlsx'  # Modify this with your actual file path

try:
    raw_data = pd.read_excel(file_path)
    print(raw_data.head())  # Display the first few rows to confirm successful load
except PermissionError:
    print("Permission Error: Unable to access the file. Make sure the file is not open in another program and check the file permissions.")
except Exception as e:
    print(f"An error occurred: {e}")

# Display the first few rows
raw_data.head()

# Display basic information about the dataset
raw_data.info()

# Check for missing values
raw_data.isnull().sum()

# Get the number of columns in the dataset
number_of_columns = raw_data.shape[1]
# Print the number of columns
print("The dataset contains", number_of_columns, "columns.")

# Get the column names
column_names = raw_data.columns

# Print the column names
print("Column Names:")
for col in column_names:
    print(col)

# Identify and Classify Columns
# Inspect column names
raw_data.columns

   Identifiant passage unique  Identifiant patient    UF   Sexe  Age(mois)  \
0                   569611249            798418724  6800  Homme       43.6   
1                   569611259            798399804  6800  Homme       40.8   
2                   569611271            798399866  6800  Homme       90.6   
3                   569611278            798405940  6800  Homme        2.0   
4                   569611033            798399772  6800  Femme       68.3   

          Date entrée         Date sortie    Médecin référent    Secteur  \
0 2023-01-01 10:44:00 2023-01-01 12:27:00   Anne Cecile CHAUX   Médecine   
1 2023-01-01 10:15:00 2023-01-01 11:18:00   Virginie LEVEQUES  Chirurgie   
2 2023-01-01 09:53:00 2023-01-01 11:04:00   Virginie LEVEQUES  Chirurgie   
3 2023-01-01 09:31:00 2023-01-01 11:05:00   Anne Cecile CHAUX   Médecine   
4 2023-01-01 23:33:00 2023-01-02 08:44:00  Charlotte THUILLER   Médecine   

   Durée séjour(h)  ... LstGrippe NbVRS LstVRS NbCovid LstCovid  TDR  \
0 

Index(['Identifiant passage unique', 'Identifiant patient', 'UF', 'Sexe',
       'Age(mois)', 'Date entrée', 'Date sortie', 'Médecin référent',
       'Secteur', 'Durée séjour(h)',
       ...
       'LstGrippe', 'NbVRS', 'LstVRS', 'NbCovid', 'LstCovid', 'TDR',
       'Resultat Grippe', 'Resultat Covid', 'Resultat VRS',
       'Resultat Streptocoque'],
      dtype='object', length=104)

In [ ]:
# Create lists to classify columns
identifiers = ['Identifiant passage unique', 'Identifiant patient']
timestamps = ['Date entrée', 'Date sortie']
attributes = ['Sexe', 'Age(mois)', 'Secteur', 'Médecin référent']
measures = ['Durée séjour(h)']
diagnostic_columns = [col for col in raw_data.columns if 'Resultat' in col or 'Lst' in col]

# Print categorized columns
print("Identifiers:", identifiers)
print("Timestamps:", timestamps)
print("Attributes:", attributes)
print("Measures:", measures)
print("Diagnostic Columns:", diagnostic_columns)

Identifiers: ['Identifiant passage unique', 'Identifiant patient']
Timestamps: ['Date entrée', 'Date sortie']
Attributes: ['Sexe', 'Age(mois)', 'Secteur', 'Médecin référent']
Measures: ['Durée séjour(h)']
Diagnostic Columns: ['LstRadiosPulmo', 'LstRadiosASP', 'LstEchos', 'LstIrm', 'LstNfp', 'LstCrp', 'LstCrpCapillaire', 'LstPct', 'LstIono', 'LstGazo', 'LstCoag', 'LstGrippe', 'LstVRS', 'LstCovid', 'Resultat Grippe', 'Resultat Covid', 'Resultat VRS', 'Resultat Streptocoque']


In [ ]:
# Data Cleaning
# Handle missing values
data_cleaned = raw_data.fillna({'Sexe': 'Unknown', 'Secteur': 'Unknown'})

# Standardize timestamp formats
data_cleaned['Date entrée'] = pd.to_datetime(data_cleaned['Date entrée'])
data_cleaned['Date sortie'] = pd.to_datetime(data_cleaned['Date sortie'])

# Check for duplicates
duplicates = data_cleaned[data_cleaned.duplicated(subset='Identifiant passage unique')]
print("Number of duplicates:", len(duplicates))

# Remove duplicates if necessary
data_cleaned = data_cleaned.drop_duplicates(subset='Identifiant passage unique')

Number of duplicates: 0


In [ ]:
# Function to process each row and extract events
def process_events(row):
    events = str(row['Histo affectations']).split('|')
    patient_id = row['Identifiant patient']
    entries = []
    for event in events:
        match = re.match(r'^(.+) \((.*?)\) du (\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2})( au (\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}))?$', event.strip())
        if match:
            entry = {
                'Identifiant patient': patient_id,
                'Personnel': match.group(1),
                'Role': match.group(2),
                'Start Time': match.group(3),
                'End Time': match.group(5) if match.group(5) else None
            }
            entries.append(entry)
        else:
            print("No match found for event:", event)
    return entries

# Load the data
raw_data = pd.read_excel('2023_01_01_ExportRechLibre.xlsx')  # Make sure to adjust the path to your input file

# Apply the function to each row and create a list of lists
processed_data = raw_data.apply(process_events, axis=1)

# Flatten the list of lists manually
flat_list = [item for sublist in processed_data for item in sublist]

# Create DataFrame from the flat list
event_data = pd.DataFrame(flat_list)

# Sort by 'Identifiant patient' and 'Start Time' for better grouping visually
event_data.sort_values(by=['Identifiant patient', 'Start Time'], inplace=True)

# Save to Excel with enhanced formatting
output_file_path = 'enhanced_output.xlsx'
with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='w') as writer:
    event_data.to_excel(writer, index=False, sheet_name='Detailed View')

    # Get the workbook and the worksheet for further manipulation
    workbook = writer.book
    worksheet = writer.sheets['Detailed View']

    # Set the column widths and format headers
    for col_num, col in enumerate(event_data.columns):
        max_len = max(event_data[col].astype(str).apply(len).max(), len(col) + 2)
        worksheet.column_dimensions[get_column_letter(col_num + 1)].width = max_len

print(f"Data has been successfully exported to {output_file_path}")

Data has been successfully exported to enhanced_output.xlsx


# Nouvelle section